<a href="https://colab.research.google.com/github/virenvaghasiya/Deep_steganography_using_CNN_and_Machine_Learning_Techniques/blob/main/Deep_steganography_using_CNN_and_Machine_Learning_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install scipy
!pip install scikit-learn
!pip install opencv-python
!pip install tensorflow
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tqdm import tqdm
import pandas as pd

from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.preprocessing import OneHotEncoder

from scipy import ndimage

import cv2



In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"virenvaghasiya","key":"e800f68187848c18a08d1f049f133da4"}'}

In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
!cp kaggle.json ~/.kaggle

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d nikhilshingadiya/tinyimagenet200

tinyimagenet200.zip: Skipping, found more recently modified local copy (use --force to force download)


**Data Preparation**

In [7]:
!unzip tinyimagenet200.zip

Archive:  tinyimagenet200.zip
replace tiny-imagenet-200/test/images/test_0.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Importing Libraries**

In [8]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tqdm import tqdm
import pandas as pd
# No need for %matplotlib inline in scripts
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, ModelCheckpoint


**Image Pre-Processing**

In [11]:
import cv2
#from keras.optimizers import Adam
files = os.listdir('tiny-imagenet-200/train')
files_te = os.listdir('tiny-imagenet-200/test/images')
x_train = np.empty((2000,64,64,3), 'uint64')
a=0
for i in range(200):
  idd = np.random.randint(0, 500, 10)
  for j in range(10):
    image = cv2.imread('tiny-imagenet-200/train/'+files[i]+'/images/'+files[i]+'_'+str(idd[j])+'.JPEG')
    x_train[a] = image
    a=a+1
x_test = np.empty((2000,64,64,3), 'uint64')
a=0
for i in range(2000):
    image = cv2.imread('tiny-imagenet-200/test/images/'+files_te[i])
    x_test[a] = image
    a=a+1
input_S = x_train[0:1000]
input_C = x_train[1000:]
input_C = input_C/255.0
input_S = input_S/255.0

In [12]:
input_C

array([[[[0.68235294, 0.70588235, 0.75686275],
         [0.57647059, 0.60392157, 0.63921569],
         [0.97254902, 0.99215686, 1.        ],
         ...,
         [0.16078431, 0.15686275, 0.14117647],
         [0.39215686, 0.38823529, 0.37254902],
         [0.27058824, 0.26666667, 0.25098039]],

        [[0.56078431, 0.58039216, 0.63921569],
         [0.57647059, 0.6       , 0.64313725],
         [0.90588235, 0.92156863, 0.94117647],
         ...,
         [0.2       , 0.19607843, 0.18039216],
         [0.31764706, 0.31372549, 0.29803922],
         [0.60392157, 0.6       , 0.58431373]],

        [[0.49803922, 0.51372549, 0.58431373],
         [0.53333333, 0.55686275, 0.60784314],
         [0.67058824, 0.68235294, 0.71372549],
         ...,
         [0.38823529, 0.38431373, 0.36862745],
         [0.56862745, 0.56470588, 0.54901961],
         [0.22745098, 0.22352941, 0.20784314]],

        ...,

        [[0.58823529, 0.58431373, 0.56862745],
         [0.59215686, 0.59215686, 0.56862745]

In [13]:
def prep_and_hide_network(input_size):
  input_message = Input(shape=(input_size))
  input_cover = Input(shape=(input_size))

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(input_message)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(input_message)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(input_message)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x = concatenate([input_cover,x])


  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (4,4), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (5,5), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  image_container = Conv2D(3, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)

  encoder = Model(inputs = [input_message, input_cover],
                  outputs = image_container)
  return encoder

In [14]:
def reveal_network(input_size, fixed=False):
  reveal_input = Input(shape=(input_size))

  input_with_noise = GaussianNoise(0.01)(reveal_input)

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(input_with_noise)
  x2 = Conv2D(10, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(input_with_noise)
  x3 = Conv2D(5, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(input_with_noise)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  x1 = Conv2D(50, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x2 = Conv2D(10, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x3 = Conv2D(5, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
  x = concatenate([x1, x2, x3])

  message = Conv2D(3, (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)

  reveal =  Model(inputs = reveal_input,
                  outputs = message)

  return reveal

In [17]:
beta = 1.0
def rev_loss(true,pred):
  loss = beta*K.sum(K.square(true-pred))
  return loss

def full_loss(true,pred):
  message_true, container_true = true[...,0:3], true[...,3:6]
  message_pred, container_pred = pred[...,0:3], pred[...,3:6]

  message_loss = rev_loss(message_true, message_pred)
  container_loss = K.sum(K.square(container_true-container_pred))

  loss = message_loss + container_loss
  return loss

In [18]:
shape = input_S.shape[1:]

input_message = Input(shape = (shape))
input_container = Input(shape = (shape))

prep_and_hide = prep_and_hide_network(shape)
reveal = reveal_network(shape)

reveal.compile(optimizer = 'adam',
              loss = rev_loss)

reveal.trainable = False

output_container = prep_and_hide([input_message, input_container])
output_message = reveal(output_container)

deep_stean = Model(inputs = [input_message, input_container],
                outputs = concatenate([output_message, output_container]))

deep_stean.compile(optimizer = 'adam',
                  loss = full_loss)

In [19]:
deep_stean.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_11            │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_9 (Functional) │ (None, 64, 64, 3)      │        293,273 │ input_layer_10[0][0],  │
│                           │                        │                │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_11             │ (None, 64, 64, 3)      │        155,938 │ functional_9[0][0]     │
│ (Functional)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_39            │ (None, 64, 64, 6)      │              0 │ functional_11[0][0],   │
│ (Concatenate)             │                        │                │ functional_9[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 449,211 (1.71 MB)

 Trainable params: 293,273 (1.12 MB)

 Non-trainable params: 155,938 (609.13 KB)